In [8]:
import os
import cryptography
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, hashes, hmac
import base64

from cryptography.hazmat.primitives import padding, serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa
import cryptography.hazmat.primitives.asymmetric as asymm

# encryption method AES-CBC-256
def Myencrypt(message, key, h):
    
    #key length check
    if len(key)<32:
        return "Error: This place is full of land mines, dragons, and dinosaurs with laser guns. Increase you key length to upgrade your armor."
    
    try:
        message = message.encode()
    except:
        pass
    
    iv = os.urandom(16)   #generate an iv
    if len(message)%16 != 0:
        padder = padding.PKCS7(128).padder()
        padded_data = padder.update(message) + padder.finalize()
        message = padded_data
    
    #HMAC
    #calling the default AES CBC mode
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    #creating an encryptor object
    encryptor = cipher.encryptor()
    #generating cipher text
    ct = encryptor.update(message) + encryptor.finalize()
    #return (ct, iv) 
    
    tag = hmac.HMAC(h, hashes.SHA256(), backend=default_backend())
    tag.update(ct)
    tag = tag.finalize()
    return (ct, iv, tag)
    


def Mydecrypt(ct, iv, tag, key, h):
    #print(tag)
    #print(ct)
    
    #Verify
    h = hmac.HMAC(h, hashes.SHA256(), backend=default_backend())
    h.update(ct)
    h = h.finalize()
    
    if h == tag:
        print("verified")
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        #creating a decryptor object
        decryptor = cipher.decryptor()
        pt = decryptor.update(ct) + decryptor.finalize()
        try:
            unpadder = padding.PKCS7(128).unpadder()
            pt = unpadder.update(pt) + unpadder.finalize()
            return pt
        except:
            return pt
    else:
        return("invalid")



# execution code    
key = os.urandom(32)
h = os.urandom(32)
m = "a secret message"
# m = "hi"
c, iv, tag = Myencrypt(m, key, h)
print("Encrypted w/ tag")
print(c, iv, tag)

print("\ndecrypted")
Mydecrypt(c, iv, tag, key, h)


Encrypted w/ tag
b'\xba8\xc2[\xd7\x00\xc8:\xddY\x19\xab("zQ' b'\xaa\xc8\xf7\xd0NV\xa5\xd2nA\xb9\x03\xb4[\xe0#' b'\x9a\xe7\xf3\x85^\xc2\xf8.\x1d\x8fF\xad)\xa6\x85"p\xa8\x8e\xb9\x90/\xe3\xce-XPx\xde\x08*\xad'

decrypted
verified


b'a secret message'

In [9]:
# file encryption algorithm
def MyfileEncrypt(filepath):
    key = os.urandom(32)
    h = os.urandom(32)
    # Read the entire file as a single byte string
    with open(filepath, 'rb') as f:
        data = f.read()

    result = Myencrypt(data, key, h)
    ext = os.path.splitext(file_path)[1]
    result += (key, h, ext)
    
    image_result = open("encrypted_image.png", 'wb') # create a writable image and write the decoding result
    image_result.write(result[0])
    
    return result

# file dencryption algorithm
def MyfileDecrypt(enc_filepath, iv, tag, key, h, ext):
    
    with open(enc_filepath, 'rb') as f:
        data = f.read()
    
    file_name = "decrypted_image" + ext
    plaintext = Mydecrypt(data, iv, tag, key, h)
    image_result = open(file_name, 'wb') # create a writable image and write the decoding result
    image_result.write(plaintext)


# ----------------------
#Execution code
file_path = os.path.abspath("hello.png")
ct, iv, tag, key, h, ext = MyfileEncrypt(file_path)
MyfileDecrypt("encrypted_image.png", iv, tag, key, h, ext)

verified
